In [14]:
###### import csv
import re
import spacy
import json
from spacy import displacy
from textblob import TextBlob
from queue import *

import re
import csv

# pre-processing
def PreProcess(senSet):
    #remove content between [ ]
    print("Pre-processing...")
    for index in range(len(senSet)):
        while senSet[index].find('[')>=0:
            i_start = senSet[index].find('[')
            i_end = senSet[index].find(']')
            s = senSet[index][i_start:i_end+2]
            senSet[index] = senSet[index].replace(s, "")

class Node:
    def __init__(self, text='', frequency=1, categ='', speaker='', start_word=0, end_word=0):
        self.text = text
        self.frequency = frequency
        self.categ = categ # category of the named entity (10 categories)
        self.speaker = speaker
        self.start_word = start_word # word index start from 0
        self.end_word = end_word
        
    def Print(self):
        print (self.name + ' ' + str(self.frequency) + ' ' + self.categ)

class Link:
    def __init__(self, Id=-1, source='', target='', sourceCateg='', targetCateg='', distance=-1.0, speaker='', frequency=1):
        self.id = Id # the id of the link is only decided by source and target, not speaker
        self.source = source
        self.target = target
        self.sourceCateg = sourceCateg
        self.targetCateg = targetCateg
        self.distance = distance # the average of all distances
        self.speaker = speaker
        self.frequency = frequency
        
    def Print(self):
        print(self.source + ' ' + self.target + ' ' + str(self.distance) + ' ' + str(self.frequency))

class Mark:
    def __init__(self, text='', categ='', start_char=0, end_char=0):
        self.text = text
        self.categ = categ # category of the named entity (10 categories)
        self.start_char = start_char
        self.end_char = end_char
        
class MarkedSen:
    def __init__(self, id, text='', speaker='', marks=[]):
        self.id = id
        self.text = text
        self.speaker = speaker
        self.marks = marks

'''class Speaker:
    def __init__(self, id=-1, name='', utterList=[]):
        self.id = id
        self.name = name
        self.utterList = utterList
        
    def Print(self):
        print(str(self.id) + ' ' + self.name)
        print(self.utterance)'''

class tempSpeaker:
    def __init__(self, name='', utterSet={}):
        self.name = name
        self.utterSet = utterSet

class tempConv:
    def __init__(self, source='', target='', utterSet={}):
        self.source = source
        self.target = target
        self.utterSet = utterSet
        
def FormatLable(label):
    if label == 'PERSON':
        return 'Person'
    elif label == 'QUANTITY':
        return 'Measure'
    elif label == 'FAC' or label == 'GPE' or label == 'LOC':
        return 'Geo'
    elif label == 'MONEY':
        return 'Unit'
    elif label == 'DATE' or label == 'TIME':
        return 'Time'
    elif label == 'NORP'or label == 'ORG':
        return 'Organization'
    else:
        return None   
    
# extract one triple from given sentence
def ExtractNER(line, index):
    #print(line['text'])
    localList = []
    
    # parse sentence
    doc = nlp(str(line['text']))

    # node related
    # basic categories
    for e in doc.ents:
        #ents = [(e.text, e.start_char, e.end_char, e.label_) ]
        #print(ents)
        
        categ = FormatLable(e.label_)
        if categ != None:
            # for node-link diagram
            if e.text in nodeSet:
                nodeSet[e.text].frequency += 1
                localList.append(nodeSet[e.text])# add current token to local set
            else:
                m_node = Node(e.text, 1, categ, line['speaker'], line['text'][:e.start_char].count(' '), line['text'][:e.end_char].count(' '))
                nodeSet[e.text] = m_node
                localList.append(nodeSet[e.text])# add current token to local set

            # for highlighted sentence
            m_mark = Mark(e.text, categ, e.start_char, e.end_char)
            lineList[index]['marks'].append(m_mark.__dict__)
            
    # emotion indicators
    for token in doc:
        #print(token.sentiment)
        testimonial = TextBlob(token.text)
        if testimonial.sentiment.polarity > 0.5 or testimonial.sentiment.polarity < -0.5:
            #for node-link diagram
            if token.text in nodeSet:
                nodeSet[token.text].frequency += 1
            else:
                if testimonial.sentiment.polarity > 0.5:
                    m_node = Node(token.text, 1, 'Positive', line['speaker'], token.i, token.i)
                else:
                    m_node = Node(token.text, 1, 'Negative', line['speaker'], token.i, token.i)
                nodeSet[token.text] = m_node
            localList.append(nodeSet[token.text])
            
            # for highlighted sentence
            if testimonial.sentiment.polarity > 0.5:
                m_mark = Mark(token.text, 'Positive', token.idx, token.idx + len(token.text))
            else:
                m_mark = Mark(token.text, 'Negative', token.idx, token.idx + len(token.text))
            lineList[index]['marks'].append(m_mark.__dict__)
    
    lineList[index]['marks'].sort(key=lambda x:x['start_char'])
    
    # link related
    # create links by traverse localList
    for token1 in localList:
        for token2 in localList:
            dist = float(token2.start_word-token1.start_word)
            if dist >0 and dist <= maxDist and token1.text != token2.text:
                #linkList.append(link.__dict__)
                link = Link(len(linkSet), token1.text, token2.text, token1.categ, token2.categ, dist, line['speaker'], 1)
                # create key by combining the source and target of link alphabetally
                key=''
                if token1.text <= token2.text:
                    key = token1.text + '-' + token2.text
                else:
                    key = token2.text + '-' + token1.text
                    
                # add link to linkSet
                if key in linkSet:
                    linkSet[key]['distance'] = float(linkSet[key]['distance'] * linkSet[key]['frequency'] + dist)/float(linkSet[key]['frequency']+1)
                    linkSet[key]['frequency'] += 1
                    link.id = linkSet[key]['id'] # this link appears before, update the id to the old one
                else:
                    linkSet[key] = link.__dict__
             
                # add link to speakerSet
                if line['speaker'] in speakerSet:
                    if key in speakerSet[line['speaker']].utterSet:# this person say it before
                        curLink = speakerSet[line['speaker']].utterSet[key]
                        speakerSet[line['speaker']].utterSet[key]['distance'] = float(curLink['distance'] * curLink['frequency']+dist)/float(curLink['frequency']+1)
                        speakerSet[line['speaker']].utterSet[key]['frequency'] += 1
                    else:# this person doesn't say it, but someone else say it before
                        speakerSet[line['speaker']].utterSet[key] = link.__dict__
                else: # no one say it before
                    speaker = tempSpeaker(line['speaker'], {})
                    speaker.utterSet[key] = link.__dict__
                    speakerSet[line['speaker']] = speaker
                
    # create links with slide window
    '''for token in doc:# go over every word in sentence
        j = len(slideWin)-1
        while j >= 0:# compare current word with the ones in the slide window
            if slideWin[j] in localList and token.text in localList:# need to fix to support phrase
                link = Link(slideWin[j], token.text, maxDist-j, 1)
                linkList.append(link.__dict__)
            j = j-1
        # update the slide window
        if i < maxDist:
            slideWin.append(token.text)
            i += 1
        else:
            #print(slideWin.get() + ' ')
            print(slideWin.pop(0))
            slideWin.append(token.text)
        #print(slideWin)'''
            
    # token level
    # ent_san = [doc[0].text, doc[0].ent_iob_, doc[0].ent_type_]
    # ent_francisco = [doc[1].text, doc[1].ent_iob_, doc[1].ent_type_]
    # print(ent_san)  # [u'San', u'B', u'GPE']
    # print(ent_francisco)  # [u'Francisco', u'I', u'GPE']

    # displacy.serve(doc, style='ent')
    
    #for token in doc:
    #    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
    #          token.shape_, token.is_alpha, token.is_stop)



# load Spacy NLP dictionary
nlp = spacy.load('en_core_web_sm')

class Line:
    def __init__(self, id=0, speaker='', text='', date='', marks=[]):
        self.id = id
        self.speaker = speaker
        self.text = text
        self.date = date
        self.marks = marks

# load data
file = open("debate.csv", "r", encoding = "ISO-8859-1")
reader = csv.reader(file)
lineList = []
utterDict = {} # dictionary for utters of each speaker
counter = 0
for item in reader:
    if counter == 0:
        counter += 1
        continue
    else:
        counter += 1
    #format sentences in item as string
    #fullP = "".join(item)
    #splitP = fullP.split(";", 3);
    #splitS = splitP[3][1:len(splitP[3])].split(".");
    #print(splitS)
    #print(item[2])
    splitS = re.split(r'(\. |\! |\? |\.\.\.|\.\" )',item[2]) #item[2].split('. ')
    splitS.append("")
    splitS = ["".join(i) for i in zip(splitS[0::2],splitS[1::2])]
    for sen in splitS:
        line = Line(len(lineList), item[1], sen, item[3], [])
        lineList.append(line.__dict__)
        #if item[1] in utterDict:
        #    utterDict[item[1]] += sen + ' '
        #else:
        #    utterDict[item[1]] = sen + ' '

file.close()
print("Total sentences: " + str(len(lineList)))

nodeSet = {}
nodeList = []# for json store
linkSet = {}
linkList = []
speakerSet = {}
speakerList = []
convSet = {}
convList = []

maxDist = 5
maxL = 0
maxI = 0

for index in range(len(lineList)):
#index = 3
    maxI = ExtractNER(lineList[index], index)

# store extracted information to json
for key in nodeSet:
    node = nodeSet[key].__dict__
    nodeList.append(node)
    
for key in linkSet:
    linkList.append(linkSet[key])
    
outputDict = {
    "nodes": nodeList,
    "links": linkList
}

i=0
for speaker1 in speakerSet:
    # create node
    ulist = []
    for k in speakerSet[speaker1].utterSet:
        ulist.append(speakerSet[speaker1].utterSet[k])
    speaker = {
        "id": i,
        "name": speaker1,
        "utterList": ulist
    }
    speakerList.append(speaker)
    i += 1
    
    # create link
    for speaker2 in speakerSet:
        if speaker1 != speaker2:
            key=''
            if speaker1 <= speaker2:
                key = speaker1 + '-' + speaker2
            else:
                key = speaker2 + '-' + speaker1
            
            for key1 in speakerSet[speaker1].utterSet:
                for key2 in speakerSet[speaker2].utterSet:
                    link1 = speakerSet[speaker1].utterSet[key1]
                    link2 = speakerSet[speaker2].utterSet[key2]
                    if link1['id'] == link2['id']: # if the two speakers have common entity pair 'link1/link2', check whether the speaker pair exists
                        if key in convSet: # if the speaker pair already exists, check whether the common entity pair 'link1/link2' exists
                            if link1['id'] in convSet[key].utterSet:# if the common entity pair 'link1/link2' also exists, increase frequency
                                convSet[key].utterSet[link1['id']]['frequency'] += 1
                            else:# if the common entity pair 'link1/link2' doesn't exist, create one, set frequency to 0, because you will visit it again?
                                convSet[key].utterSet[link1['id']] = link1
                                convSet[key].utterSet[link1['id']]['frequency'] = 1
                        else:# if the speaker pair doesn't exist, create one, put current common entity pair into the utterSet
                            conv = tempConv(speaker1, speaker2, {})
                            conv.utterSet[link1['id']] = link1
                            conv.utterSet[link1['id']]['frequency'] = 1
                            convSet[key] = conv
    
for key in convSet:
    utterList = []
    for key_link in convSet[key].utterSet:
        utterList.append(convSet[key].utterSet[key_link])
    conv = {
        "source": convSet[key].source,
        "target": convSet[key].target,
        "utterList": utterList
    }
    convList.append(conv)
    
#sort
for speaker in speakerList:
    speaker['utterList'].sort(key=lambda x:x['frequency'], reverse=True)

for conv in convList:
    conv['utterList'].sort(key=lambda x:x['frequency'], reverse=True)
    
speakerDict = {
    "nodes": speakerList,
    "links": convList
}
    
    
#with open('node.json', 'w') as f:
#    f.write(json.dumps(outputDict, indent=4, separators=(',', ':')))

#with open('highlight.json', 'w') as f:
#    f.write(json.dumps(lineList, indent=4, separators=(',', ':')))

with open('speakers.json', 'w') as f:
    f.write(json.dumps(speakerDict, indent=4, separators=(',', ':')))

print("Total nodes: " + str(len(nodeList)))
print("Total links: " + str(len(linkList)))
print("Total speakers: " + str(len(speakerList)))
#print(utterDict['QUESTION'])

Total sentences: 5816
Total nodes: 820
Total links: 557
Total speakers: 10


In [65]:
import re
s = "aidfs? df! dfsdf. dfsdfs... sdfsdf.\"akekrw"
s = re.split(r'(\. |\! |\? |\.\.\.|\.\")',s)
s.append("")
s = ["".join(i) for i in zip(s[0::2],s[1::2])]
print(s)

['aidfs? ', 'df! ', 'dfsdf. ', 'dfsdfs...', ' sdfsdf."', 'akekrw']


In [36]:
#compare two strings alphabetically
token1 = "trums"
token2 = "trump"
if token1 <= token2:
    print(token1 + '-' + token2)
else:
    print(token2 + '-' + token1)

trump-trums


In [44]:
a = [1,2,3,4,5]
print(a[::-1])

[5, 4, 3, 2, 1]
